<a href="https://colab.research.google.com/github/1900690/YOLO-dataset-sprit/blob/main/AI_dataset_sprit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown fastlabelでエクスポートしたzipファイルをアップロード

#ライブラリをインポート(機能を使えるようにする)
import shutil
import os
from google.colab import files
import math


#すでにフォルダがある場合は消す
if os.path.exists("/content/datasets"):
  shutil.rmtree("/content/datasets")

#データをアップロード
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
#データを解凍
shutil.unpack_archive('/content/'+file_name, '/content/')
#zipを消す
os.remove('/content/'+file_name)

Saving fastlabeldata.zip to fastlabeldata.zip


In [20]:
#@markdown 画像を学習用と検証用とテスト用に分割

#@markdown ※初めに全体をtrainとvalidに分割、分割したtrainからさらにtestを分割する
#ライブラリをインポート
import shutil
import os
from sklearn.model_selection import train_test_split
import math

#分割元のパスを作成
originals='/content/original'
annotations='/content/yolo/annotations'

#分割先のパスを作成
train_images = '/content/datasets/train/images'
train_labels = '/content/datasets/train/labels'
valid_images = '/content/datasets/valid/images'
valid_labels = '/content/datasets/valid/labels'
test_images = '/content/datasets/test/images'
test_labels = '/content/datasets/test/labels'

#分割先のファイルを作成
if os.path.exists('/content/datasets/'):
  shutil.rmtree('/content/datasets/')
if not os.path.exists('/content/datasets/'):
  os.makedirs(train_images)
  os.makedirs(train_labels)
  os.makedirs(valid_images)
  os.makedirs(valid_labels)
  os.makedirs(test_images)
  os.makedirs(test_labels)


#フォルダの中のファイルのリストを作成
read_files_annotations = os.listdir(annotations)
read_files_annotations.sort()
read_files_originals= os.listdir(originals)
read_files_originals.sort()


valid分割割合=0.218#@param {type:"slider", min:0, max:1, step:0.001}
test分割割合=0.016#@param {type:"slider", min:0, max:1, step:0.001}

#分割割合に合わせてファイルのリストを分割
annotations_train_test, annotations_valid, originals_train_test, originals_valid = train_test_split(read_files_annotations,read_files_originals,test_size=valid分割割合)
annotations_train, annotations_test, originals_train, originals_test = train_test_split(annotations_train_test,originals_train_test,test_size=test分割割合)


#画像とアノテーションを分ける
for filename in annotations_train:
  shutil.copy( annotations+"/"+filename,train_labels)
for filename in annotations_valid:
  shutil.copy( annotations+"/"+filename,valid_labels)
for filename in annotations_test:
  shutil.copy( annotations+"/"+filename,test_labels)

for filename in originals_train:
  shutil.copy( originals+"/"+filename,train_images)
for filename in originals_valid:
  shutil.copy( originals+"/"+filename,valid_images)
for filename in originals_test:
  shutil.copy( originals+"/"+filename,test_images)

#最大公約数を計算
greatest_common_divisor=math.gcd(sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)),sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images)))

#分割されたファイル数を表示
print("train/imagesは",sum(os.path.isfile(os.path.join(train_images, name)) for name in os.listdir(train_images)),"枚")
print("train/labelsは",sum(os.path.isfile(os.path.join(train_labels, name)) for name in os.listdir(train_labels)),"枚")
print("valid/imagesは",sum(os.path.isfile(os.path.join(valid_images, name)) for name in os.listdir(valid_images)),"枚")
print("valid/labelsは",sum(os.path.isfile(os.path.join(valid_labels, name)) for name in os.listdir(valid_labels)),"枚")
print("test/imagesは",sum(os.path.isfile(os.path.join(test_images, name)) for name in os.listdir(test_images)),"枚")
print("test/labelsは",sum(os.path.isfile(os.path.join(test_labels, name)) for name in os.listdir(test_labels)),"枚")
print("最大公約数は",greatest_common_divisor,"<-この数はbatchを決める際に必要、2の倍数で８くらいがちょうどいい")

train/imagesは 984 枚
train/labelsは 984 枚
valid/imagesは 280 枚
valid/labelsは 280 枚
test/imagesは 16 枚
test/labelsは 16 枚
最大公約数は 8 <-この数はbatchを決める際に必要


#学習データに関する詳細ファイルを作成

In [21]:
#以下の★の部分を学習目的に応じて書き換える
%%writefile /content/datasets/data.yaml

#★検出する項目の数だけ-の後に項目名を記載する
names:
- OK
- NG

#★検出する項目の数を記載
nc: 2

#ここは触らない
test: ../test/images
train: ../train/images
val: ../valid/images

Writing /content/datasets/data.yaml


In [22]:
#@markdown 作ったファイルをダウンロード
import shutil
import os
from google.colab import files

shutil.make_archive('/content/datasets', 'zip', '/content/datasets')

#ダウンロード
files.download("/content/datasets.zip")


'/content/datasets.zip'